#### Question 1:

Consider the two data files (users.csv, transactions.csv). Users file has the following fields:
<ol><li>	UserID</li>
<li>	EmailID</li>
<li>	NativeLanguage</li>
<li>	Location</li></ol>

Transactions file has the following fields:
<ol><li>	Transaction_ID</li>
<li>	Product_ID</li>
<li>	UserID</li>
<li>	Price</li>
<li>	Product_Description</li></ol>

By making use of Spark Core (i.e. without using Spark SQL) find out:
<ol><li>	Count of unique locations where each product is sold. </li>
<li>	Find out products bought by each user.</li>
<li>	Total spending done by each user on each product.</li></ol>


<b>Amit Kumar Sharma<b><br>
PGID: 71721079

In [1]:
import os

In [3]:
os.getcwd()
os.chdir("/Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data")

In [4]:
user = sc.textFile('file:///Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/users.csv')
transaction = sc.textFile('file:///Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/transactions.csv')

In [5]:
user.count()

30

In [6]:
transaction.count()

15

In [7]:
transaction.collect()

[u'1,1004,19,129,whatchamacallit',
 u'2,1001,10,99,thingamajig',
 u'3,1004,17,129,whatchamacallit',
 u'4,1001,9,99,thingamajig',
 u'5,1003,3,89,gadget',
 u'6,1002,19,149,gizmo',
 u'7,1002,30,149,gizmo',
 u'8,1002,26,149,gizmo',
 u'9,1001,22,99,thingamajig',
 u'10,1003,6,89,gadget',
 u'11,1004,1,129,whatchamacallit',
 u'12,1004,2,129,whatchamacallit',
 u'13,1005,5,199,doohickey',
 u'14,1004,7,129,whatchamacallit',
 u'15,1002,16,149,gizmo']

In [8]:
user.take(20)

[u'1,user1@company.com,ES,MX',
 u'2,user4@domain.com,EN,US',
 u'3,user5@company.com,FR,FR',
 u'4,user9@site.org,HI,IN',
 u'5,user12@service.io,EN,CA',
 u'6,user17@website.net,FR,FR',
 u'7,user21@company.com,FR,FR',
 u'8,user25@company.com,FR,FR',
 u'9,user27@school.edu,ES,MX',
 u'10,user31@website.net,EN,CA',
 u'11,user36@website.net,FR,FR',
 u'12,user39@domain.com,FR,FR',
 u'13,user41@company.com,ES,MX',
 u'14,user45@domain.com,HI,IN',
 u'15,user48@site.org,ES,MX',
 u'16,user53@school.edu,EN,US',
 u'17,user57@school.edu,ES,MX',
 u'18,user59@website.net,HI,IN',
 u'19,user64@school.edu,EN,US',
 u'20,user67@domain.com,HI,IN']

Join user and transaction data by userId and stored joined RDD as temp.

In [9]:
u = user.map(lambda x : x.split(",")).map(lambda x: (x[0],x))
t = transaction.map(lambda x : x.split(",")).map(lambda x: (x[2],x))
temp = u.join(t).map(lambda x : x[1][0]+x[1][1]).map(lambda x : ",".join(x))

In [10]:
temp.take(5)

[u'19,user64@school.edu,EN,US,1,1004,19,129,whatchamacallit',
 u'19,user64@school.edu,EN,US,6,1002,19,149,gizmo',
 u'1,user1@company.com,ES,MX,11,1004,1,129,whatchamacallit',
 u'5,user12@service.io,EN,CA,13,1005,5,199,doohickey',
 u'9,user27@school.edu,ES,MX,4,1001,9,99,thingamajig']

In [11]:
temp.count()

15

In [12]:
temp.collect()

[u'19,user64@school.edu,EN,US,1,1004,19,129,whatchamacallit',
 u'19,user64@school.edu,EN,US,6,1002,19,149,gizmo',
 u'1,user1@company.com,ES,MX,11,1004,1,129,whatchamacallit',
 u'5,user12@service.io,EN,CA,13,1005,5,199,doohickey',
 u'9,user27@school.edu,ES,MX,4,1001,9,99,thingamajig',
 u'10,user31@website.net,EN,CA,2,1001,10,99,thingamajig',
 u'17,user57@school.edu,ES,MX,3,1004,17,129,whatchamacallit',
 u'22,user71@domain.com,ES,MX,9,1001,22,99,thingamajig',
 u'26,user85@service.io,HI,IN,8,1002,26,149,gizmo',
 u'3,user5@company.com,FR,FR,5,1003,3,89,gadget',
 u'7,user21@company.com,FR,FR,14,1004,7,129,whatchamacallit',
 u'30,user99@website.net,EN,US,7,1002,30,149,gizmo',
 u'16,user53@school.edu,EN,US,15,1002,16,149,gizmo',
 u'2,user4@domain.com,EN,US,12,1004,2,129,whatchamacallit',
 u'6,user17@website.net,FR,FR,10,1003,6,89,gadget']

### Count the unique locations where each product is sold?

In [13]:
temp.map(lambda x: x.split(",")[3]).distinct().take(20)

[u'MX', u'CA', u'FR', u'US', u'IN']

In [14]:
temp.map(lambda x: x.split(",")[3]).distinct().count()

5

### Find out products bought by each user?

In [15]:
temp.map(lambda x: x.split(",")).map(lambda x: (int(x[0]),x[5])).reduceByKey(lambda x, y: x+","+y).sortByKey().collect()

[(1, u'1004'),
 (2, u'1004'),
 (3, u'1003'),
 (5, u'1005'),
 (6, u'1003'),
 (7, u'1004'),
 (9, u'1001'),
 (10, u'1001'),
 (16, u'1002'),
 (17, u'1004'),
 (19, u'1004,1002'),
 (22, u'1001'),
 (26, u'1002'),
 (30, u'1002')]

In [16]:
groupedByUser = temp.map(lambda x: x.split(",")).map(lambda x: (x[0],x[5])).groupByKey()
groupedByUser.map(lambda x : (int(x[0]),list(x[1]))).sortByKey().collect()

[(1, [u'1004']),
 (2, [u'1004']),
 (3, [u'1003']),
 (5, [u'1005']),
 (6, [u'1003']),
 (7, [u'1004']),
 (9, [u'1001']),
 (10, [u'1001']),
 (16, [u'1002']),
 (17, [u'1004']),
 (19, [u'1004', u'1002']),
 (22, [u'1001']),
 (26, [u'1002']),
 (30, [u'1002'])]

### Total spending done by each user on each product?

In [17]:
temp.map(lambda x: x.split(",")).map(lambda x: ((int(x[0]),int(x[5])),int(x[7]))).reduceByKey(lambda x, y: int(x)+int(y)).sortByKey().collect()

[((1, 1004), 129),
 ((2, 1004), 129),
 ((3, 1003), 89),
 ((5, 1005), 199),
 ((6, 1003), 89),
 ((7, 1004), 129),
 ((9, 1001), 99),
 ((10, 1001), 99),
 ((16, 1002), 149),
 ((17, 1004), 129),
 ((19, 1002), 149),
 ((19, 1004), 129),
 ((22, 1001), 99),
 ((26, 1002), 149),
 ((30, 1002), 149)]